**Multiclass Fish Image Classification using Pre-trained models**

**EfficientNetB0**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torchvision.models import efficientnet_b0
from torch.utils.data import DataLoader
import os
from PIL import Image
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, confusion_matrix

In [ ]:
# set the device
#This means you’ll run your model on GPU

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
#transformation for training data
train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(224, scale = (0.8, 1.0)), #random zoom + crop
        transforms.RandomHorizontalFlip(), #  flip the image (right ---> left)
        transforms.RandomRotation(15), # random rotation clockwise or counter clockwise
        transforms.ToTensor()   # #rescale to [0,1]
    ])

In [ ]:
 #for test and val
# data preprocessing

test_val_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)), #resize piexel same as training
        transforms.ToTensor() #convert image to tensor
    ]
)

In [ ]:
# load the datasets
train_datasets = datasets.ImageFolder('/content/drive/MyDrive/fish_image_classification_project_5/images.cv_jzk6llhf18tm3k0kyttxz/data/train', transform = train_transform)
test_datasets = datasets.ImageFolder('/content/drive/MyDrive/fish_image_classification_project_5/images.cv_jzk6llhf18tm3k0kyttxz/data/test', transform = test_val_transform)
val_datasets = datasets.ImageFolder('/content/drive/MyDrive/fish_image_classification_project_5/images.cv_jzk6llhf18tm3k0kyttxz/data/val', transform = test_val_transform)

In [ ]:
# create the dataloader
train_data_loader = DataLoader(train_datasets, batch_size = 5, shuffle = True)
test_data_loader = DataLoader(test_datasets, batch_size = 5, shuffle = True)
val_data_loader = DataLoader(val_datasets, batch_size = 5, shuffle = True)


In [ ]:
# load the pre-trained  model EfficientNetB0
model = models.efficientnet_b0(pretrained = True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 122MB/s]


In [ ]:
model.classifier[1] = nn.Linear(in_features=1280, out_features=11)
model = model.to(device)

In [ ]:
#loss function and optimizer

cl = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(),lr = 0.001)

In [ ]:
# train the model

epochs = 25

for i in range(epochs):
  model.train()
  total_loss = 0.0 # total running loss
  total = 0 #  #  Total number of images evaluated
  correct = 0 #  calculate how many predictions correct
  for img, tar in train_data_loader:
    img, tar = img.to(device), tar.to(device)
    optimizer.zero_grad()
    outputs = model(img)
    loss = cl(outputs, tar)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    _, preds = torch.max(outputs, 1)
    correct += (preds == tar).sum().item()
    total += tar.size(0)

  print(f"Epoch [{i + 1}/{epochs}], loss {total_loss:.4f}")


Epoch [1/25], loss 902.8445
Epoch [2/25], loss 444.5125
Epoch [3/25], loss 361.9542
Epoch [4/25], loss 345.1612
Epoch [5/25], loss 329.6713
Epoch [6/25], loss 305.8383
Epoch [7/25], loss 279.1798
Epoch [8/25], loss 286.0884
Epoch [9/25], loss 271.3515
Epoch [10/25], loss 279.0989
Epoch [11/25], loss 270.8975
Epoch [12/25], loss 264.9159
Epoch [13/25], loss 279.1433
Epoch [14/25], loss 267.3674
Epoch [15/25], loss 284.7060
Epoch [16/25], loss 280.6456
Epoch [17/25], loss 251.2245
Epoch [18/25], loss 250.8101
Epoch [19/25], loss 240.2469
Epoch [20/25], loss 257.0794
Epoch [21/25], loss 258.5691
Epoch [22/25], loss 241.7442
Epoch [23/25], loss 278.9647
Epoch [24/25], loss 271.6884
Epoch [25/25], loss 271.1959


In [ ]:
#Evaluation on validation data
model.eval()

all_preds = []
all_tars = []

with torch.no_grad():
  for img, tar in val_data_loader:
    img, tar = img.to(device), tar.to(device)
    outputs = model(img)
    _, preds = torch.max(outputs, 1)


    all_preds.extend(preds.cpu().numpy())
    all_tars.extend(tar.cpu().numpy())


  #accuracy
  accuracy = accuracy_score(all_tars, all_preds)
  print(f"val_accuracy: {accuracy*100 :.2f}%")

  #precision
  precision = precision_score(all_tars , all_preds, average = 'macro')
  print(f"val_precision: {precision*100 :.2f}%")

  # recall
  recall = recall_score(all_tars , all_preds, average = 'macro')
  print(f"val_recall: {recall*100 :.2f}%")

  #f1 score
  f1score = f1_score(all_tars, all_preds, average = 'macro')
  print(f"val f1_score: {f1score *100:.2f}%")

  # Calculate confusion matrix
  cm = confusion_matrix(all_tars, all_preds)
  print('confusion matrix')
  print(cm)

val_accuracy: 97.53%
val_precision: 94.08%
val_recall: 95.40%
val f1_score: 94.56%
confusion matrix
[[180   6   0   0   0   0   0   1   0   0   0]
 [  3   7   0   0   0   0   0   0   0   0   0]
 [  0   0 105   0   0   0   0   0   0   0   0]
 [  0   0   0  94   0   0   0   0   0   0   0]
 [  0   0   0   0  97   0   0   0   0   0   0]
 [  0   0   0   0   0  90   0   0   0   0   0]
 [  0   0   0   0   0   0 113   0   0   0   0]
 [  1   0   0   0   0   0   0  96   0   0   0]
 [  0   0   0   0   0   0   0   0 100   0   0]
 [  0   0   2   1   0   5   0   1   7  85   0]
 [  0   0   0   0   0   0   0   0   0   0  98]]


In [ ]:
# Evaluation on test data

model.eval()

all_preds = []
all_tars = []

with torch.no_grad():
  for img, tar in test_data_loader:
    img, tar = img.to(device), tar.to(device)
    outputs = model(img)
    _, preds = torch.max(outputs, 1)
    all_preds.extend(preds.cpu().numpy())
    all_tars.extend(tar.cpu().numpy())


# accuracy
accuracy = accuracy_score(all_tars, all_preds)
print(f"Test Accuracy: {accuracy *100:.2f}%")

# precision
precision = precision_score(all_tars, all_preds, average = 'macro')
print(f"Test precision: {precision *100:.2f}%")

#recall
recall = recall_score(all_tars, all_preds, average = 'macro')
print(f"Test recall: {recall *100:.2f}%")


#f1 score
f1score = f1_score(all_tars, all_preds, average = 'macro')
print(f"Test f1_score: {f1score *100:.2f}%")


# Calculate confusion matrix
cm = confusion_matrix(all_tars, all_preds)
print('confusion matrix')
print(cm)


Test Accuracy: 97.77%
Test precision: 92.57%
Test recall: 96.09%
Test f1_score: 93.55%
confusion matrix
[[500  19   0   0   0   0   0   1   0   0   0]
 [  3  10   0   0   0   0   0   0   0   0   0]
 [  0   0 298   0   0   0   0   0   0   0   0]
 [  1   0   0 298   0   0   3   3   0   0   0]
 [  0   0   1   1 284   0   0   0   0   0   0]
 [  0   0   0   0   0 291   0   0   0   0   0]
 [  0   0   0   0   0   0 273   0   0   0   0]
 [  2   0   1   3   0   0   0 321   0   0   0]
 [  0   0   0   0   0   0   0   0 289   0   0]
 [  0   0   5   0   2  19   0   2   4 261   0]
 [  0   0   0   0   0   0   0   1   0   0 291]]


In [ ]:
# evaluate the train data

model.eval()

all_preds = []
all_tars = []


with torch.no_grad():
  for img, tar in train_data_loader:
    img, tar = img.to(device), tar.to(device)
    outputs = model(img)
    _,preds = torch.max(outputs, 1)

    all_preds.extend(preds.cpu().numpy())
    all_tars.extend(tar.cpu().numpy())


  # accuracy
accuracy = accuracy_score(all_tars, all_preds)
print(f"Train Accuracy: {accuracy* 100:.2f}%")

  #precision
precision = precision_score(all_tars, all_preds, average = 'macro')
print(f"Train precision: {precision*100:.2f}%")

  #recall
recall = recall_score(all_tars, all_preds, average = 'macro')
print(f"Train recall:{recall*100:.2f}%")

#f1 score
f1score = f1_score(all_tars, all_preds, average = 'macro')
print(f"Train f1_score: {f1score *100:.2f}%")

# Calculate confusion matrix
cm = confusion_matrix(all_tars, all_preds)
print('confusion matrix')
print(cm)

Train Accuracy: 98.55%
Train precision: 93.98%
Train recall:98.57%
Train f1_score: 95.28%
confusion matrix
[[1054   40    1    0    0    0    0    1    0    0    0]
 [   1   29    0    0    0    0    0    0    0    0    0]
 [   0    0  569    0    0    0    0    0    0    0    0]
 [   0    0    0  564    0    0    0    1    0    0    1]
 [   1    0    5    0  567    0    0    0    0    0    0]
 [   0    0    0    0    0  578    0    1    0    0    0]
 [   0    0    0    1    0    0  570    0    0    0    0]
 [   1    0    2    2    0    0    0  532    0    0    1]
 [   0    0    0    0    0    0    0    0  576    0    0]
 [   1    0    4    0    1   17    0    0    7  517    0]
 [   0    0    0    0    0    0    0    1    0    0  579]]


In [ ]:
# save the model

torch.save(model.state_dict(), 'efficientnetb0_fish_model.pth')

In [ ]:
from google.colab import files
files.download('efficientnetb0_fish_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>